# Codigo para capturar las imagenes de las emociones


### Josué Montalván Zavala

##### Captura de frames para guardar y clasificar

In [6]:
import numpy as np
import cv2 as cv
import math 

haar_cascade_path = cv.data.haarcascades + 'haarcascade_frontalface_alt.xml'
rostro_cascade = cv.CascadeClassifier(haar_cascade_path)

cap = cv.VideoCapture(0)
i = 0 
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    rostros = rostro_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in rostros:
        # Extraer el rostro de la imagen en escala de grises
        rostro_gray = gray[y:y+h, x:x+w]
        rostro_gray = cv.resize(rostro_gray, (48, 48), interpolation=cv.INTER_AREA)
        cv.imwrite(f'C:/Users/crazu/Documents/Inteligencia artificial/Emociones/Tristeza/t{i}.png', rostro_gray)
        
    cv.imshow('rostros', frame)
    i += 1
    k = cv.waitKey(1)
    if k == 27:  # Presiona 'Esc' para salir
        break

cap.release()
cv.destroyAllWindows()


 # Escritura del xml de entrenamiento/reconocimiento
    

In [ ]:
import cv2
import os
import numpy as np
import time

def obtenerModelo(method,facesData,labels):
    
    if method == 'EigenFaces': emotion_recognizer = cv2.face.EigenFaceRecognizer_create()
    # Entrenando el reconocedor de rostros
    print("Entrenando ( "+method+" )...")
    inicio = time.time()
    emotion_recognizer.train(facesData, np.array(labels))
    tiempoEntrenamiento = time.time()-inicio
    print("Tiempo de entrenamiento ( "+method+" ): ", tiempoEntrenamiento)

    # Almacenando el modelo obtenido
    emotion_recognizer.write("modelo"+method+".xml")

dataPath = '/Users/crazu/Documents/Inteligencia artificial/Emociones/' #Cambia a la ruta donde hayas almacenado Data
emotionsList = os.listdir(dataPath)
print('Lista de emociones: ', emotionsList)

labels = []
facesData = []
label = 0

for nameDir in emotionsList:
    emotionsPath = dataPath + '/' + nameDir

    for fileName in os.listdir(emotionsPath):
        #print('Rostros: ', nameDir + '/' + fileName)
        labels.append(label)
        facesData.append(cv2.imread(emotionsPath+'/'+fileName,0))
        #image = cv2.imread(emotionsPath+'/'+fileName,0)
        #cv2.imshow('image',image)
        #cv2.waitKey(10)
    label = label + 1

obtenerModelo('EigenFaces',facesData,labels)
#obtenerModelo('FisherFaces',facesData,labels)
#obtenerModelo('LBPH',facesData,labels)

# Reconocimiento de las emociones

In [ ]:
# import cv2
import os
import numpy as np

method = 'EigenFaces'
if method == 'EigenFaces':
    emotion_recognizer = cv2.face.EigenFaceRecognizer_create()

emotion_recognizer.read('modelo' + method + '.xml')


dataPath = '/Users/crazu/Documents/Inteligencia artificial/Emociones/' 
imagePaths = os.listdir(dataPath)
print('imagePaths=', imagePaths)

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')

while True:

    ret, frame = cap.read()
    if ret == False:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    auxFrame = gray.copy()

    nFrame = cv2.hconcat([frame, np.zeros((480, 300, 3), dtype=np.uint8)])

    faces = faceClassif.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        rostro = auxFrame[y:y + h, x:x + w]
        rostro = cv2.resize(rostro, (48, 48), interpolation=cv2.INTER_CUBIC)
        result = emotion_recognizer.predict(rostro)

        cv2.putText(frame, '{}'.format(imagePaths[result[0]]), (x, y - 5), 1, 1.3, (255, 255, 0), 1, cv2.LINE_AA)

        # EigenFaces
        if method == 'EigenFaces':
            if result[1] < 5700:
                cv2.putText(frame, '{}'.format(imagePaths[result[0]]), (x, y - 25), 2, 1.1, (0, 255, 0), 1, cv2.LINE_AA)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                nFrame = cv2.hconcat([frame])
            else:
                cv2.putText(frame, 'No identificado', (x, y - 20), 2, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                nFrame = cv2.hconcat([frame, np.zeros((480, 300, 3), dtype=np.uint8)])

    cv2.imshow('nFrame', nFrame)
    k = cv2.waitKey(1)
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()


# Renombrar imagenes

In [ ]:
import os
from PIL import Image

# Directorio de entrada y salida
input_dir = 'C:\\Users\\crazu\\Downloads\\triste'
output_dir = 'C:/Users/crazu/Documents/Carpeta IA/IA-Josu-Montalv-n/Emociones/EEnojo'

# Crear la carpeta de salida si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Función para generar un nuevo nombre de archivo
def generate_new_filename(old_filename, prefix='BW_', suffix=''):
    name, ext = os.path.splitext(old_filename)
    return f"{prefix}{name}{suffix}{ext}"

# Obtener lista de archivos en el directorio de entrada
for filename in os.listdir(input_dir):
    # Comprobar que el archivo es una imagen
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        # Ruta completa del archivo
        file_path = os.path.join(input_dir, filename)
        # Abrir la imagen
        with Image.open(file_path) as img:
            # Convertir a blanco y negro
            bw_img = img.convert('L')
            # Generar nuevo nombre de 


# Renombrar y en blanco y negro 

In [ ]:
import cv2
import os

def resize_and_convert_to_bw(input_folder, output_folder, size=(48, 48), prefix='BW_', suffix=''):
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Contador para el número de imágenes procesadas
    num_images_processed = 0

    # Función para generar un nuevo nombre de archivo
    def generate_new_filename(old_filename):
        name, ext = os.path.splitext(old_filename)
        return f"{prefix}{name}{suffix}{ext}"

    # Recorrer todos los archivos en la carpeta de entrada
    for filename in os.listdir(input_folder):
        if filename.lower().endswith((".jpg", ".png", ".jpeg")):
            img_path = os.path.join(input_folder, filename)
            img = cv2.imread(img_path)

            if img is not None:
                resized_img = cv2.resize(img, size)
                bw_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
                new_filename = generate_new_filename(filename)
                output_path = os.path.join(output_folder, new_filename)
                cv2.imwrite(output_path, bw_img)
                num_images_processed += 1
            else:
                print(f"No se pudo leer la imagen {filename}")
        else:
            print(f"El archivo {filename} no es una imagen soportada, se omitirá")

    print(f"Proceso finalizado: {num_images_processed} imágenes redimensionadas, convertidas a blanco y negro, y guardadas en {output_folder}")

input_folder = 'C:\\Users\\crazu\\Downloads\\triste'  # Cambia esto a la ruta de tu carpeta de entrada
output_folder = 'C:/Users/crazu/Documents/Carpeta IA/IA-Josu-Montalv-n/Emociones/EEnojo'  # Cambia esto a la ruta de tu carpeta de salida

resize_and_convert_to_bw(input_folder, output_folder)


# Deteccion de emociones

In [ ]:
import cv2 as cv
import os 

faceRecognizer = cv.face.LBPHFaceRecognizer_create()
faceRecognizer.read("C:\Users\\crazu\\Documents\\Inteligencia artificial\\modeloEigenFaces.xml")
faces = ["Enojo","Feliciadad","Sorpresa","Tristeza"]
cap = cv.VideoCapture(0)
rostro = cv.CascadeClassifier('C:/Users/crazu/Documents/Inteligencia artificial/haarcascade_frontalface_alt.xml')
while True:
    ret, frame = cap.read()
    if ret == False: break
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cpGray = gray.copy()
    rostros = rostro.detectMultiScale(gray, 1.3, 3)
    for(x, y, w, h) in rostros:
        frame2 = cpGray[y:y+h, x:x+w]
        frame2 = cv.resize(frame2,  (48,48), interpolation=cv.INTER_CUBIC)
        result = faceRecognizer.predict(frame2)
        cv.putText(frame, '{}'.format(result), (x,y-20), 1,3.3, (255,255,0), 1, cv.LINE_AA)
        if result[1] < 100:
            cv.putText(frame,'{}'.format(faces[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv.LINE_AA)
            cv.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
        else:
            cv.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv.LINE_AA)
            cv.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2) 
    cv.imshow('frame', frame)
    k = cv.waitKey(1)
    if k == 27:
        break
cap.release()
cv.destroyAllWindows()

In [1]:
import cv2
import os
import numpy as np

# ----------- Métodos usados para el entrenamiento y lectura del modelo ----------
method = 'EigenFaces'

if method == 'EigenFaces':
    emotion_recognizer = cv2.face.EigenFaceRecognizer_create()

emotion_recognizer.read('modelo'+method+'.xml')
# --------------------------------------------------------------------------------

dataPath = 'C:\\Users\\crazu\\Documents\\Inteligencia artificial\\Emociones'  # Cambia a la ruta donde hayas almacenado Data
imagePaths = os.listdir(dataPath)
print('imagePaths=', imagePaths)

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')

while True:
    ret, frame = cap.read()
    if ret == False: break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    auxFrame = gray.copy()

    nFrame = cv2.hconcat([frame, np.zeros((480, 300, 3), dtype=np.uint8)])

    faces = faceClassif.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        rostro = auxFrame[y:y + h, x:x + w]
        rostro = cv2.resize(rostro, (48, 48), interpolation=cv2.INTER_CUBIC)
        result = emotion_recognizer.predict(rostro)

        cv2.putText(frame, '{}'.format(result), (x, y - 5), 1, 1.3, (255, 255, 0), 1, cv2.LINE_AA)

        if result[1] < 5700:
            cv2.putText(frame, '{}'.format(imagePaths[result[0]]), (x, y - 25), 2, 1.1, (0, 255, 0), 1, cv2.LINE_AA)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            nFrame = cv2.hconcat([frame])
        else:
            cv2.putText(frame, 'No identificado', (x, y - 20), 2, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            nFrame = cv2.hconcat([frame, np.zeros((480, 300, 3), dtype=np.uint8)])

    cv2.imshow('nFrame', nFrame)
    k = cv2.waitKey(1)
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()


imagePaths= ['Enojo', 'Felicidad', 'Sorpresa', 'Tristeza']
